Script para gerar dados falsos simulando grãos de soja de diferentes estados e fazendas.

In [1]:
import numpy as np
import pymongo

In [2]:
db = pymongo.MongoClient()['nosqlba']

Doc exemplo:

```python
{
    "_id": ObjectId(...),
    "origem": {
        "estado": "Goiás",
        "fazenda": "Fazenda Aqui Se Faz Soja"
    },
    "safra": "2016/2017",
    "cor": "Marrom",
    "dimensoes": {
        "x": 5.0394,
        "y": 4.8743
    },
    "peso": 1.1579
    "agrotoxicos": [
        "veneno forte 1",
        "veneno fraco 2",
        "remédio x"
    ]   
}
```

In [3]:
toxicos = {
    'toxicos1': ['veneno_{}'.format(i) for i in range(1,5)] + ['remedio_{}'.format(l) for l in ['a', 'b', 'x']],
    'toxicos2': ['veneno_{}'.format(i) for i in [1,1,1,1,1,1,1,1,2,2,2,3,3,3,3]] + ['remedio_x', 'remedio_y'],
    'toxicos3': ['remedio_{}'.format(l) for l in ['a', 'a', 'a', 'b', 'x', 'y']]*3 + ['veneno_{}'.format(i) for i in range(2,5)],
    'toxicos4': ['remedio_{}'.format(l) for l in ['y']*9 + ['c']*3 + ['b']*3],
}

In [4]:
estados = ['Goiás']*5 + ['Paraná']*3 + ['Mato Grosso']*2
origens = {
    "Goiás": {
        "fazendas": ["Estância da Soja"]*4 + ["Fazenda Só Soja Já!"]*7 + ["Fazenda Del Busso"]*5 + ["Fazenda Aqui Se Faz Soja"]*10 + ["Flor do Goiás"]*12 + ["Soja Export"]*8,
        "carac": {
            "Estância da Soja": [(5, 0.2), (4, 0.1), (1.15, 0.15), (4, 'toxicos1')],
            "Fazenda Só Soja Já!": [(5.5, 0.4), (4.8, 0.12), (1.3, 0.13), (2, 'toxicos1')],
            "Fazenda Del Busso": [(4.3, 0.45), (3.5, 0.5), (1.3, 0.2), (2, 'toxicos3')],
            "Fazenda Aqui Se Faz Soja": [(7.3, 0.4), (6, 0.4), (1.7, 0.2), (7, 'toxicos3')],
            "Flor do Goiás": [(5, 0.8), (5, 0.3), (1.15, 0.1), (3, 'toxicos1')],
            "Soja Export": [(5, 0.05), (5, 0.05), (1.15, 0.03), (4, 'toxicos2')],
        },
        'cores': ["Marrom"]*50 + ["Verde"]*30 + ["Mesclado"]*20
    },
    "Mato Grosso": {
        "fazendas": ["Soja Grossa Plantation"]*10 + ["Soja Faz Bem Para Você SA"]*6 + ["Soja Da Terra"]*3,
        "carac": {
            "Soja Grossa Plantation": [(9, 0.23), (8.9, 0.17), (1.75, 0.2), (3, 'toxicos1')],
            "Soja Faz Bem Para Você SA": [(6, 0.4), (5, 0.6), (1.3, 0.11), (8, 'toxicos2')],
            "Soja Da Terra": [(3, 0.6), (2.6, 0.5), (1.5, 0.14), (1, 'toxicos4')]
        },
        'cores': ["Marrom"]*20 + ["Verde"]*20 + ["Mesclado"]*60
    },
    "Paraná": {
        "fazendas": ["Fazenda Para na Soja"]*9 + ["Estância O Sol Já Nasceu"]*2,
        "carac": {
            "Fazenda Para na Soja": [(4, 0.1), (4, 0.2), (0.9, 0.1), (5, 'toxicos2')],
            "Estância O Sol Já Nasceu": [(4.2, 0.4), (4.2, 0.5), (1, 0.17), (3, 'toxicos3')],
        },
        'cores': ["Marrom"]*10 + ["Verde"]*70 + ["Mesclado"]*20
    }
}

In [5]:
%%time
np.random.seed(433)
safra = '2015/2016'
for i in range(6):
    if i >= 3:
        safra = '2016/2017'
    updates = []
    for j in range(100000):
        estado = np.random.choice(estados)
        fazenda = np.random.choice(origens[estado]['fazendas'])
        cor = np.random.choice(origens[estado]['cores'])
        caracs = origens[estado]['carac'][fazenda]
        x = np.random.normal(caracs[0][0], caracs[0][1])
        y = np.random.normal(caracs[1][0], caracs[1][1])
        peso = np.random.normal(caracs[2][0], caracs[2][1])
        if cor == 'Marrom':
            peso *= 1.1
        if i < 3 and fazenda == "Fazenda Del Busso":
            x *= np.random.normal(3, 1)
            y *= np.random.normal(3, 1)
            peso *= np.random.normal(5, 1)
        tox = list(np.random.choice(toxicos[caracs[3][1]], size=caracs[3][0]))

        doc = {
            'origem': {
                'estado': estado,
                'fazenda': fazenda
            },
            'safra': safra,
            'cor': cor,
            'dimensoes': {
                'x': x,
                'y': y
            },
            'peso': peso,
            'toxicos': tox
        }
        
        updates.append(pymongo.InsertOne(doc))
    db.soja.bulk_write(updates)
    print('bulk', i+1)



bulk 1
bulk 2
bulk 3
bulk 4
bulk 5
bulk 6
CPU times: user 1min 32s, sys: 2.57 s, total: 1min 34s
Wall time: 1min 36s
